# Forecast API endpoint


In [ ]:
import xgboost as xgb

In [ ]:
import json

import altair as alt
import httpx
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from shapely import geometry
from shapely.ops import unary_union
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

In [ ]:
tete_aga = ["2363", "2189", "2190", "2191", "2202", "2192", "2193", "2194", "2203", "2195", "2196", "2197", "2198", "2199", "2200", "2201", "2204", "2205", "2206", "2208", "2209", "2207", "2210", "2211", "2212", "2213", "2214", "2215", "2216", "2359", "2360", "2361", "2362", "2364", "2365", "2366", "2367", "2368", "2369", "2370", "2371", "2372", "2409", "2410", "2411", "2412", "2413", "2414", "2415", "2454", "2416", "2452", "2453", "2455", "2456", "2459", "2457", "2458", "2460", "2461", "2462", "2463", "2464"]

tete_aga = list(map(int, tete_aga))

# API Auth

In [ ]:
base_url = "http://localhost:8081/"

In [ ]:
client = httpx.Client(base_url=base_url)

In [ ]:
r = client.post(
    "auth/token",
    data={"username": os.environ["USERNAME"], "password": os.environ["PASSWORD"]},
)

In [ ]:
token = json.loads(r.text)["access_token"]
headers = {"Authorization": f"Bearer {token}"}

# Get agricultural areas geoms from the API

In [ ]:
polygons = []
for wb_id in tete_aga:
    try:
        r = client.get("aoi/", params=dict(id=wb_id), headers=headers)
        res = json.loads(r.text)
        polygons.append(geometry.shape(res["features"][0]["geometry"]))
    except:
        continue

In [ ]:
# get the bbox for all the ag areas
box = unary_union(polygons).bounds


In [ ]:
box

# Get 7 Months Forecast from DB 

start_datetime should be at least 3 month earlier than endtime because it is using the weather data of last 3 months.


In [ ]:
start_datetime = "2020-07-01"
end_datetime = "2020-10-10"

In [ ]:
r = client.get(
        "forecast/",
        params=dict(
            bbox=box,
            aoi_id=tete_aga,
            start_datetime=start_datetime,
            end_datetime=end_datetime,
            limit=1000,
        ),
        headers=headers,
        timeout=60,
    )
    

In [ ]:
plt.plot(json.loads(r.text)['forecast'])